De nodige imports voor epic 3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dotenv
from sqlalchemy import create_engine, text

dotenv.load_dotenv()

# mochten we de suprise lib gebruiken -> optimaler
# from surprise import Dataset, Reader, SVD
# from surprise.model_selection import cross_validate

False

Alle (gecleande) data in een dataframe

In [2]:
df = pd.read_csv('Data/?.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'Contact.csv'

In [ ]:
X_df, y_df = df.drop(columns=["attended"]), df["attended"]

X_df.shape, y_df.shape

Nu moet je de data splitsen in training en test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X_df, y_df, test_size=0.1, random_state=42
)

X_val_df, X_test_df, y_val_df, y_test_df = train_test_split(
    X_test_df, y_test_df, test_size=0.25, random_state=42
)

In [ ]:
X_train_df.shape, X_val_df.shape, X_test_df.shape

Dit zorgt ervoor dat de code numeriek is (anti-error), ontbrekende values behandeld etc

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

single_cols = []  # 1 unique value

df_objects = X_train_df.select_dtypes(include="object").columns

label_encoder = LabelEncoder()
X_df_encoded = X_df.copy()
for col in df_objects:
    X_df_encoded[col] = label_encoder.fit_transform(X_df_encoded[col])
    X_train_df[col] = label_encoder.transform(X_train_df[col])
    X_val_df[col] = label_encoder.transform(X_val_df[col])
    X_test_df[col] = label_encoder.transform(X_test_df[col])

# Numeric or float
assert X_train_df.select_dtypes(include="object").shape[1] == 0

imp = SimpleImputer(missing_values=np.nan, strategy="mean")
imp.fit(X_df_encoded)
X_train_df = imp.transform(X_train_df)
X_val_df = imp.transform(X_val_df)
X_test_df = imp.transform(X_test_df)




from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_df = scaler.fit_transform(X_train_df)
X_val_df = scaler.transform(X_val_df)
X_test_df = scaler.transform(X_test_df)

# NaN
assert np.isnan(X_train_df).sum() == 0
X_train_df.shape, X_val_df.shape, X_test_df.shape, y_train_df.shape

In [ ]:
assert np.isnan(X_train_df).any() == False
assert np.isnan(X_test_df).any() == False

Nu maken we het model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy

model = Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(X_train_df.shape[1],)))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(32, activation="relu"))  
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer="RMSProp", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
epochs = 50  # Editable
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=int(epochs / 15),  # Editable
    restore_best_weights=True,
)

hist = model.fit(
    X_train_df,
    y_train_df,
    epochs=epochs,
    batch_size=32,  # Editable
    validation_data=(X_val_df, y_val_df),
    callbacks=[early_stopping],
)


Evalueer het model en plot het

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_df, y_test_df)

y_pred = model.predict(X_test_df)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

from sklearn.metrics import classification_report
report = classification_report(y_test_df, y_pred_binary)
print(report)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test_df, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Didn't attend", "Attended"])
disp.plot()

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.set_style("darkgrid")
sns.set_palette("Set2")

sns.lineplot(
    x=range(len(hist.history["accuracy"])),
    y=hist.history["accuracy"],
    label="accuracy",
    ax=ax[1],
)
sns.lineplot(
    x=range(len(hist.history["val_accuracy"])),
    y=hist.history["val_accuracy"],
    label="val_accuracy",
    ax=ax[1],
)

ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Accuracy")

plt.show()

Slaag het model op!

In [ ]:
model.save("Data/e3.keras")
print("Saved")